In [1]:
# Basics
import numpy as np
import pandas as pd
import os
import pickle
import h5py

/Users/anders1991/miniconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
with open("./data/partition.pkl", 'rb') as f:
    partition = pickle.load(f)

# Load labels

In [5]:
target = pd.read_csv("./data/ERU_Scores_Ids_5-Scans_Validity-0_VisuallyScored.csv")
labels = target.set_index("StId").to_dict()["ERU.M2"]

In [6]:
label_converter = {0: 0.0, 1: 0.0, 2: 0.03, 3: 0.155, 4: 0.38, 5: 0.63, 6: 0.88} # combine 0+1 as 0 = no emph in scan, 1 = no emph in region
labels = {key: label_converter[val] for key, val in labels.items()}

# Load data

In [20]:
from data_gen import DataGenerator

In [21]:
timesteps = 1
batch_size = 5
downsample = 2

In [22]:
predGen = DataGenerator("single_valid", batch_size=batch_size, timesteps=timesteps, 
                         channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=True)
predGen = predGen.generate(labels, partition["valid"])

# Load model

In [23]:
from keras.models import load_model

In [24]:
model = load_model("cnn1/single/epoch_13-valacc_0.73.hdf5")

In [52]:
X, y = predGen.__next__()
y_ = model.predict_on_batch(X)

comp = np.empty((5,2))
comp[:,0] = np.reshape(y*100, (5))
comp[:,1] = np.reshape(y_*100, (5))
print(comp)

[[ 0.          2.4748466 ]
 [15.5         7.83679724]
 [38.          4.8092742 ]
 [ 3.         -1.22850931]
 [ 3.          0.03874952]]
